Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04_model = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_10_04_model",
monitors = ["a.1", "a.2", "a.3", "a.4", "a.5", "a.6", "a.7", "bp", "bpC"],
model=m_10_04_model, output_format=:mcmcchain);

Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
# Result rethinking
rethinking = "
   mean   sd  5.5% 94.5% n_eff Rhat
      mean   sd  5.5% 94.5% n_eff Rhat
a[1] -0.74 0.27 -1.17 -0.31  3838    1
a[2] 11.02 5.53  4.46 21.27  1759    1
a[3] -1.05 0.28 -1.50 -0.61  3784    1
a[4] -1.05 0.27 -1.48 -0.62  3761    1
a[5] -0.74 0.27 -1.18 -0.32  4347    1
a[6]  0.21 0.27 -0.23  0.66  3932    1
a[7]  1.81 0.39  1.19  2.46  4791    1
bp    0.84 0.26  0.42  1.26  2586    1
";

Describe the draws

In [8]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74278526 0.26678070 0.0042181732 0.0045875929 1000
a.2 10.86817275 5.32018607 0.0841195277 0.1362432565 1000
a.3 -1.05785679 0.28077014 0.0044393657 0.0045973650 1000
a.4 -1.05205759 0.28344767 0.0044817012 0.0045159032 1000
a.5 -0.74001654 0.26392292 0.0041729878 0.0037529267 1000
a.6  0.21425808 0.26892633 0.0042520986 0.0042022909 1000
a.7  1.80957039 0.38438691 0.0060776908 0.0061772115 1000
 bp  0.84217010 0.26117117 0.0041294788 0.0041118436 1000
bpC -0.13331086 0.29705098 0.0046967884 0.0045521272 1000

Quantiles:
        2.5%       25.0%       50.0%       75.0%       97.5%   
a.1 -1.27004375 -0.92156575 -0.7419935 -0.559001250 -0.23517983
a.2  3.92502000  6.72371500  9.7922250 13.996800000 23.97023000
a.3 -1.60668225 -1.24780250 -1.0573250 -0.862009500 -0.51654235
a.4 -1.63129375 -1.23615750 -1.04626

End of `10/m10.02s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*